# Backward method

In the `Bias` layer implementation, the formulas for the derivatives were relatively simple, and the complexity relied on how to use the framework and in the understanding of the difference between the derivative of the input and the one of the parameters.

`Linear` layer's backward method requires teh calculation of $\frac{dE}{dy}$ and $\frac{dE}{dw}$. In terms of the framework the implementation is very similar to the `Bias` layer, but the formulas of the derivatives are more complex.

First we'll assume there's only one input example $x$ to make it simpler, the we'll generalize to a batch of $N$ examples.


## $dE / dx$

Let's start with $\frac{dE}{dx}$. This is symmetrical to $\frac{dE}{dw}$,but easier to grasp conceptually.

We'll think this derivative by scenario, from the simplest to the most complex, increasing the input and output dimensions.


### 1 input, 1 output

When both the input and output are 1D then $x \in R$ and $w \in R$, they're scalars. Then $\frac{dE}{dy}$ is also a scalar, and following the Chain Rule:

$\frac{dE}{dx} = \frac{dE}{dy} \frac{dy}{dx} = \frac{dE}{dy} \frac{d(wx)}{dx} = \frac{dE}{dy} w$



### I inputs, 1 output

With $I$ inputs and 1 output $x$ is a vector with $I$ values, i.e. $x \in R^I$, then $w \in R^I$ is also a vector with $I$ values. Then we can think the output as the matrix product between $w$ and $x$

$y = x . w = \sum_{i=1}^I x_i w_i$

We have one partial derivative for each input: $\frac{dE}{dx_j}$. Given that $\frac{dE}{dy}$ is still a scalar (there's only one output) and applying the Chain Rule, we can calculate this derivative:

$
\frac{dE}{dx_j} 
= \frac{dE}{dy} \frac{dy}{dx_j} \\
= \frac{dE}{dy} \frac{d (\sum_{i=1}^I w_i x_i)}{dx_j} \\
= \frac{dE}{dy} \sum_{i=1}^I \frac{d (w_i x_i) }{dx_j} \\
= \frac{dE}{dy} \frac{d (w_j x_j) }{dx_j} \\
= \frac{dE}{dy} w_j 
$

Then $\frac{dE}{dx_j} = \frac{dE}{dy} w_j$. We can generalize this definition and calculate the gradient with respect to the whole vector $x$ as:

$\frac{dE}{dx} = \frac{dE}{dy} w$


#### Notes

1. It's great that the same definition of $\frac{dE}{dx}$ works in both scenarios, being with $1$ input or with an arbitrary amount of $I$ inputs.
1. It's important to consider that in this context we cant think of $\frac{dE}{dy}$ as a constant, since it's values were calculated previously.
1. We could obtain $\frac{dy}{dx}$ without taking into consideration the network error, and then get $\frac{dE}{dx}$ applying the Chain Rule $\frac{dE}{dx} =\frac{dE}{dy} \frac{dy}{dx}$. We're doing everythiong at the same time to be clearer in the context of the `backward` method of a network.



### I inputs, O outputs

Again, let's go for one of the input's derivatives, i.e., $\frac{dE}{dx_j}$:

$
\frac{dE}{dx_j} = \frac{dE}{dy} \frac{dy}{dx_j} 
$

In this case $y$ is a vector, so we have to add the contribution of each element of $y$ to the Chain Rule:

$
\frac{dE}{dx_j} 
= \frac{dE}{dy} \frac{dy}{dx_j}  
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{dy_i}{dx_j}
$

We know that $y_i$ is the dot product between the column $i$ of $w$ with the input $x$, according to the matrix multiplication definition, then:

$
\frac{dE}{dx_j} 
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{dy_i}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{d(w_{:,i} \cdot x)}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{d(\sum_{k=1}^I w_{k,i} x_k)}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} ( \sum_{k=1}^I \frac{d (w_{k,i} x_k)}{dx_j} ) \\
= \sum_{i=1}^O \frac{dE}{dy_i} w_{j,i}
$

Now, $\sum_{i=1}^O \frac{dE}{dy_i} w_{j,i}$ is simply the dot product between the column $i$ of $w$ ($w_{:,i}$) and $\frac{dE}{dy}$. Then we can write:

$
\frac{dE}{dx_j}  = \frac{dE}{dy} \cdot w_{:,i}
$

Generalizing to the entire vector $x$, if $\frac{dE}{dx_j}$ is the product between tho vectors, where $j$ is the column of $w$, we can write $\frac{dE}{dx}$ as a product between the $\frac{dE}{dy}$ vector and the entire $w$ matrix:

$
\frac{dE}{dx} = w \frac{dE}{dy}
$

In this case again, the order matters. $w$ has size $I \times O$ and $\frac{dE}{dy}$ has size $O$, then $w \frac{dE}{dy}$ has size $I$ (the same as $x$)




### Batch implementation

To implement the derivative for an example batch we can iterate over each and calculate the derivatives as we did before. Alternatively we can rewrite the derivative to make it work for a batch of $N$ examples (then, of $N$ vectors of derivatives, both for the input and for the output).

In the batch implementation of $\frac{dE}{dx}$ we have $x$ as a matrix of size $N \times I$, then $\frac{dE}{dx}$ also is. At the same time as $\frac{dE}{dy}$ is next layer's $\frac{dE}{dx}$, $\frac{dE}{dy}$ is a matrix of size $N \times O$.

Given that, we can't multiply $w \in R^{I \times O}$ by $\frac{dE}{dy} \in R^{N \times O}$. In this case you can verify that the correct formula is $\frac{dE}{dy} w^T$, since when multiplying a matrix of size $N \times O$ by one of size $O \times I$ ($w^T$), we get a matrix of size $N \times I$: the same size as $x$:

$
\frac{dE}{dx} = \frac{dE}{dy} w^T
$

## $dE/dw$

For the gradient of the error with respect of $w$, we'll also assume at first that there's only one input example $x$, and we'll go from the simplest to the more complex scenarios.



### 1 input, 1 output

This is the simplest scneario, and it's symmetrical to $\frac{dE}{dx}$:

$
\frac{dE}{dw} = \frac{dE}{dw} \frac{dw}{dx} = \frac{dE}{dw} \frac{d (wx)}{dw} = \frac{dE}{dy} x
$



### I inputs, 1 output

Now we have $I$ inputs and 1 output.

$y = x . w = \sum_{i=1}^I x_i w_i$

As $w$ has $I$ elements there's a partial derivative for each value of $w$: $\frac{dE}{dw_j}$. Keep in mind that $\frac{dE}{dy}$ is still a scalar (there's only one output), so applying the Chain Rule we can calculate this derivative:

$
\frac{dE}{dw_j} 
= \frac{dE}{dy} \frac{dy}{dw_j} \\
= \frac{dE}{dy} \frac{d \sum_{i=1}^I w_i x_i }{dw_j} \\
= \frac{dE}{dy} \sum_{i=1}^I \frac{d (w_i x_i) }{dxw_j} \\
= \frac{dE}{dy} \frac{d (w_j x_j) }{dw_j} \\
= \frac{dE}{dy} x_j
$

Then $\frac{dE}{dw_j} = \frac{dE}{dy} x_j$. We can generalize this definition and calculate the gradient with respect to the entire vector $x$ as:

$\frac{dE}{dw} = \frac{dE}{dy} x$

Again, this case is **symmetrical** with $x$, since $\frac{dE}{dx} = \frac{dE}{dy} w$.



### I inputs, O outputs

In this case having $O$ outputs, we'll have to find the derivative of each weight for each input $i$ for each output $j$. We'll lose the previous symmetry, but it'll be recovered in the batch version.

Given that, we'll find $\frac{dE}{dw_{i,j}}$. Applying the Chain Rule:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{dy}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}} 
$

As $y$ is a vector, we'll have to add for each value to apply the Chain Rule:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}}
= \sum_{k=1}^O \frac{dE}{dy_k} \frac{d(xw)_k}{dw_{i,j}} 
$

As $y_k$ only depends on $w_{i,j}$ if $j=k$, i.e. if we're calculating the output for the column $k$, then:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}}
= \frac{dE}{dy_j} \frac{d(xw)_j}{dw_{i,j}} 
$

By the matrix multiplication definition, $(xw)_j = \sum_{l=1}^O x_l w_{l,j}$, so we multiply $x$ for the column $j$ of $w$. Replacing the values:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy_j} \frac{d(xw)_j}{dw_{i,j}} \\
= \frac{dE}{dy_j} \frac{d(\sum_{l=1}^O x_l w_{l,j})}{dw_{i,j}} \\
= \frac{dE}{dy_j} \sum_{l=1}^O \frac{d (x_l w_{l,j}) }{dw_{i,j}}
$

As $w_{i,j}$ is one particular weight of $w$, of the entire sum only remains the term that contains it: $\frac{d (x_i w_{i,j})}{w_{i,j}} = x_i$. Replacing the values:

$
\frac{dE}{dw_{i,j}} 
= \frac{dE}{dy_j} \sum_{l=1}^O \frac{d (x_l w_{l,j})}{dw_{i,j}} \\
= \frac{dE}{dy_j} \frac{d(x_i w_{i,j})}{d w_{i,j}} \\ 
= \frac{dE}{dy_j} x_i
$



### Vector expression

The previous expression is helpful, but we should use a `for` loop with `i` and `j` indexes over the entire `w` matrix. Instead we can generalize by observing the pattern of the $\frac{dE}{dw}$ matrix:

$
\frac{dE}{dw} = \left(
\begin{matrix} 
    \frac{dE}{dy_1} x_1 & \frac{dE}{dy_2} x_1 & \dots & \frac{dE}{dy_O} x_1 \\
    \frac{dE}{dy_1} x_2 & \frac{dE}{dy_2} x_2 & \dots & \frac{dE}{dy_O} x_2 \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{dE}{dy_1} x_I & \frac{dE}{dy_2}  x_I & \dots & \frac{dE}{dy_O}  x_I \\
\end{matrix}
\right) = x \otimes \frac{dE}{dy}
$

Where $\otimes$ is the [outer product](https://en.wikipedia.org/wiki/Outer_product) between two vectors. With `numpy` the [`outer`](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) function allows this kind of operation without the need of loops.

Keep in mind that the outer product is *not* commutative: if $a$ and $b$ have sizes $p$ and $q$, then $ a \otimes b$ has size $p \times q$ and $b \otimes a$ has size $q \times p$. Given that, as $\frac{dE}{dw}$ must have size $I \times O$, we have to calculate $x \otimes\frac{dE}{dy}$ and not $\frac{dE}{dy} \otimes x$.



## Batch calculation

When we have a batch of $N$ examples, $x$ has size $N \times I$, $w$ has size $I \times O$, and $\frac{dE}{dy}$ has size $N \times O$.

As before with $b$, to calculate $\frac{dE}{dw}$ we need to add the gradient that contributes with each example $x_i$, then:

$
\frac{dE}{dw} = \sum_{i=1}^{n} x_{i,:} \otimes \frac{dE}{dy_{i,:}}
$

Where $x_{i,:}$ is $x$'s $i$ row, i.e. the $i^{th}$ example (`numpy`'s equivalent would be `x[i,:]`)

For example, for $N=2$ we can verify:

$
\frac{dE}{dw} = x_{1,:}  \otimes \frac{dE}{dy_{1,:}} + x_{1,:}  \otimes \frac{dE}{dy_{2,:}}  \\
=  \left(
\begin{matrix}
    \frac{dE}{dy_{1,1}} x_{1,1} + \frac{dE}{dy_{2,1}} x_{2,1} & \frac{dE}{dy_{1,2}} x_{1,1} + \frac{dE}{dy_{2,2}} x_{2,1} & \dots & \frac{dE}{dy_{1,O}} x_{1,1} + \frac{dE}{dy_{2,O}} x_{2,1} \\
    \frac{dE}{dy_{1,1}} x_{1,2} + \frac{dE}{dy_{2,1}} x_{2,2} & \frac{dE}{dy_{1,2}} x_{1,2} + \frac{dE}{dy_{2,2}} x_{2,2} & \dots & \frac{dE}{dy_{1,O}} x_{1,2} + \frac{dE}{dy_{2,O}} x_{2,2} \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{dE}{dy_{1,1}} x_{1,I} + \frac{dE}{dy_{2,1}} x_{2,I} & \frac{dE}{dy_{1,2}} x_{1,I} + \frac{dE}{dy_{2,2}} x_{2,I} & \dots & \frac{dE}{dy_{1,O}} x_{1,I} + \frac{dE}{dy_{2,O}} x_{2,I} \\
\end{matrix}
\right) \\
 = x^t \frac{dE}{dy}
$

This is valid for any $N$. We can confirm this identity given the sizes: multiplying $x^t$ (size $I \times N$) by $\frac{dE}{dy}$ (size $N \times O$), we obtain a matrix of size $I \times O$, same as $w$, just the size $\frac{dE}{dw}$ must have!.

Then we can see the symmetry between both derivatives;:

$
\frac{dE}{dw} = \frac{dy}{dw} \frac{dE}{dy} = x^t \frac{dE}{dy} \\
\frac{dE}{dx}= \frac{dy}{dx} \frac{dE}{dy} = \frac{dE}{dy} w
$
